In [ ]:
# This code creates coronal slice image, 3D segmentation image, and 3D lung Mask if the segmentations and label maps were created correctly


import os
import pandas as pd
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import nrrd
from PIL import Image, ImageDraw
import os
import vtk
from mayavi import mlab

# Keep Case Numbers undefined if the previous code blocks were ran, otherwise you can specify a case number to choose to run through the image creation program
# case_numbers = [17]

def find_files(folder_path):
    vx3_file = None
    mask_file = None
    label_file = None
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file == "Vx3.nrrd":
                vx3_file = os.path.join(root, file)
            elif file.endswith(".mask.nii.gz"):
                mask_file = os.path.join(root, file)
            elif file.endswith("label.nrrd"):
                label_file = os.path.join(root, file)
        
        if vx3_file and mask_file and label_file:
            break  # All files are found, exit the loop
    
    return vx3_file, mask_file, label_file

base_folder = "z:/D-Images/MESAL-3/2-Checked-Done-Temp/Done/"

for case_number in case_numbers:
    case_folder_prefix = f"Case-{case_number}-"
    
    for folder in os.listdir(base_folder):
        if folder.startswith(case_folder_prefix):
            folder_path = os.path.join(base_folder, folder)
            vx3_file, mask_file, label_file = find_files(folder_path)

            if vx3_file and mask_file and label_file:
                print("All files were found for", folder)
                print("Vx3 File:", vx3_file)
                print("Mask File:", mask_file)
                print("Label File:", label_file)
                # 200 FINAL WORKING CODE FOR IMAGE GENERATION FOR ONE CASE
                ############################################################################################################################################################
                # QC IMAGE STUFF
                # Casename = "H-14479_JH110088_SPI-JH110088-PBV_SPIROMICS_EXPIRATION_PBV__0.5__QR40__RESEARCH__A_80KV_Qr40d_3_53062428"
                Vx3Path = vx3_file
                labelpath = label_file
                maskpath = mask_file
                output_path = 'z:/Q- Figures/T32-Poster/Nate/Experiment/'
                filename = output_path + "-QC-Image.png"
                final_filename = output_path + "-QC-Image(2CM).png"
                Vx3_path = Vx3Path
                
                # 3D Screenshot Stuff check for repeats
                nrrd_file_path = labelpath
                segmentation_path = labelpath
                case_number = folder
                output_path_seg = 'z:/Q- Figures/T32-Poster/Nate/Experiment/'  # Change this to your desired output directory
                output_nrrd_file_path = "z:/Q- Figures/T32-Poster/Nate/Experiment/"

                # These filenames are just to have the program run all at the same time
                Aseed_filename_base = case_number + "Aseed"
                Vseed_filename_base = case_number + "Vseed"

                vtk_filename_aseed = f"{Aseed_filename_base}.vtk"
                vtk_filename_vseed = f"{Vseed_filename_base}.vtk"
                output_filename_aseed = output_path_seg + vtk_filename_aseed
                output_filename_vseed = output_path_seg + vtk_filename_vseed
                screenshot_filename = "Model.png"


                models_folder = os.path.join(os.path.dirname(output_path), "QC-Images")
                os.makedirs(models_folder, exist_ok=True)

                # Load the image using nibabel to see image information
                data, header = nrrd.read(Vx3_path)

                # Print the metadata
                print("Data Shape:", data.shape)
                print("Data Type:", data.dtype)
                print("Header:", header)

                # Load the 3D image
                Vx3 = sitk.ReadImage(Vx3_path)
                print("Reading Vx3...")
                # Get the size of the image in x, y, and z dimensions
                size_x, size_y, size_z = Vx3.GetSize()
                spacing = Vx3.GetSpacing()
                y_index = Vx3.GetSize()[1] // 2
                print("Spacing:", spacing)

                # Extract the middle plane along the y-dimension
                slice_2d = Vx3[:,y_index,:]
                slice_array = sitk.GetArrayFromImage(slice_2d)

                # Set the Window level
                window_level_min = -1024  # Adjust these values according to your preference
                window_level_max = 650

                # Apply Window Level to image
                windowed_image = sitk.IntensityWindowing(slice_2d, window_level_min, window_level_max, 0, 255)
                windowed_slice_array = sitk.GetArrayFromImage(windowed_image)

                #Show Vx3 with correct window level
                plt.imshow(windowed_slice_array, cmap="gray",origin='lower')
                plt.axis("off")
                plt.show()

                # Save the Corrected Vx3 .PNG to output path
                output_file_path = os.path.join(models_folder, filename)
                plt.imsave(filename, windowed_slice_array, cmap="gray", origin='lower')
                print("Image saved at:", output_file_path)

                # Load the image
                image_path = filename
                output_path_png = output_path + "/QC-Images/" + case_folder_prefix + "QC-Image(2CM).png"
                image = Image.open(image_path)

                # Determine line position
                line_start_x = 65 # 65 pixels from the left
                line_start_y = image.height - 35 # 35 pixels from the bottom

                # Calculate line end position
                line_end_x = line_start_x + int(20 / 0.38)  # 10 cm converted to pixels
                line_end_y = line_start_y

                # Draw the white line
                draw = ImageDraw.Draw(image)
                draw.line([(line_start_x, line_start_y), (line_end_x, line_end_y)], fill='white', width=2)
                # # Save the modified image
                image.save(output_path_png)

                print("Scale bar added and image saved.")

                #############################################################################################################################################################


                def update_camera_info(obj, evt):
                    camera = mlab.gcf().scene.camera
                    print("Camera Position:", camera.position)
                    print("Camera Focal Point:", camera.focal_point)
                    print("Camera View Up:", camera.view_up)


                # This line creates Models Forlder if not already made
                if not os.path.exists(output_path_seg):
                    os.makedirs(output_path_seg)


                # Load the .nrrd segmentation for Vseed
                nrrd_reader_vseed = vtk.vtkNrrdReader()
                nrrd_reader_vseed.SetFileName(segmentation_path)
                nrrd_reader_vseed.Update()

                vtk_image_data_vseed = vtk.vtkImageData()
                vtk_image_data_vseed.DeepCopy(nrrd_reader_vseed.GetOutput())

                # Define scalar value for "Vseed" segment
                vseed_scalar_value = 2

                # Create a Marching Cubes filter for "Vseed" segment
                contour_vseed = vtk.vtkMarchingCubes()
                contour_vseed.SetInputData(vtk_image_data_vseed)
                contour_vseed.SetValue(0, vseed_scalar_value)
                contour_vseed.Update()

                # Write the "Vseed" mesh to a .vtk file
                vtk_writer_vseed = vtk.vtkPolyDataWriter()
                vtk_writer_vseed.SetFileName(output_filename_vseed) 
                vtk_writer_vseed.SetInputData(contour_vseed.GetOutput())
                vtk_writer_vseed.Write()

                print("Vseed Segmentation conversion complete.")

                # Read the .nrrd file
                data, header = nrrd.read(nrrd_file_path)

                data[data == 2] = 0 
                output_nrrd_file_path = output_path + "Segmentation(Aseed).nrrd"
                # Write the modified data and header to a new .nrrd file
                nrrd.write(output_nrrd_file_path, data, header)
                print("Aseed.nrrd created!")
                # Load the .nrrd segmentation for Aseed
                nrrd_reader_aseed = vtk.vtkNrrdReader()
                nrrd_reader_aseed.SetFileName(output_nrrd_file_path)
                nrrd_reader_aseed.Update()

                vtk_image_data_aseed = vtk.vtkImageData()
                vtk_image_data_aseed.DeepCopy(nrrd_reader_aseed.GetOutput())

                # Define scalar value for "Aseed" segment
                aseed_scalar_value = 1

                # Create a Marching Cubes filter for "Aseed" segment
                contour_aseed = vtk.vtkMarchingCubes()
                contour_aseed.SetInputData(vtk_image_data_aseed)
                contour_aseed.SetValue(0, aseed_scalar_value)
                contour_aseed.Update()

                # Write the "Aseed" mesh to a .vtk file
                vtk_writer_aseed = vtk.vtkPolyDataWriter()
                vtk_writer_aseed.SetFileName(output_filename_aseed) 
                vtk_writer_aseed.SetInputData(contour_aseed.GetOutput())
                vtk_writer_aseed.Write()

                print("Aseed Segmentation Conversion Complete")

                #CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
                screenshot_output = output_path_seg + case_number + screenshot_filename

                mlab.figure(bgcolor=(1, 1, 1))
                srcA = mlab.pipeline.open(output_filename_aseed)
                srcV = mlab.pipeline.open(output_filename_vseed)

                # Create a surface visualization of the data with blue color and 50% opacity
                surfaceA = mlab.pipeline.surface(srcA, color=(0, 151/255, 206/255), opacity=1)
                surfaceV = mlab.pipeline.surface(srcV, color=(216/255, 101/255, 79/255), opacity=1)

                # Find Segmentation Center
                model = sitk.ReadImage(segmentation_path)
                origin = model.GetOrigin()
                dimensions = model.GetSize()
                direction = model.GetDirection()
                spacing = (0.38, 0.38, 0.38)
                distance = np.array(spacing) * np.array(dimensions)
                center = np.array(origin) + np.array(distance) / 2


                x, y, z = center[0], center[1], center[2]
                ypt = y + 600

                camera_focal_point = np.array([x, y, z])
                camera_position = np.array([x,ypt,z])

                # Set the camera parameters
                # Write code to calculate this
                mlab.gcf().scene.camera.position = [x, ypt, z]
                mlab.gcf().scene.camera.focal_point = [x, y, z]
                mlab.gcf().scene.camera.view_up = [0, 0, 1]

                # Attach the callback to the interaction event
                mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

                magnification_factor = 5
                mlab.savefig(screenshot_output, magnification=magnification_factor)


                print("Process Completed")

                ###################################################################################################################################################################

                # 2 to fix (nearly done)
                ## THIS CODE WORKS TO MAKE A MODEL AND POSITION IT AT A FAVORABLE ANGLE TO TAKE AN IMAGE OF THE MAYAVI REPRESENTATION OF THE MODEL

                # QC IMAGE STUFF
                # Casename = "H-14479_JH110088_SPI-JH110088-PBV_SPIROMICS_EXPIRATION_PBV__0.5__QR40__RESEARCH__A_80KV_Qr40d_3_53062428"

                # 3D Screenshot Stuff check for repeats
                nrrd_file_path = labelpath
                segmentation_path = labelpath
                # output_nrrd_file_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/Aseed.nrrd"

                # Path to the input NIfTI file
                input_nifti_path_mask = maskpath
                # Path to save the output NRRD file
                output_nrrd_path_mask = 'z:/Q- Figures/T32-Poster/Nate/Experiment/Case-30-Lung-Mask.nrrd'
                # Load the NIfTI image
                nifti_image = sitk.ReadImage(input_nifti_path_mask)
                # Create a writer for NRRD format
                nrrd_writer = sitk.ImageFileWriter()
                nrrd_writer.SetFileName(output_nrrd_path_mask)
                # Convert the NIfTI image to a Numpy array
                nifti_array = sitk.GetArrayFromImage(nifti_image)
                # Convert the Numpy array back to a SimpleITK image
                nrrd_image = sitk.GetImageFromArray(nifti_array)
                # Copy information (metadata) from NIfTI image to NRRD image
                nrrd_image.CopyInformation(nifti_image)
                # Write the NRRD image
                nrrd_writer.Execute(nrrd_image)
                print("Conversion complete.")

                masknrrdpath = output_nrrd_path_mask

                # These filenames are just to have the program run all at the same time
                rLung_filename_base = case_number + "Lungs"
                Aseed_filename_base = case_number + "Aseed"

                vtk_filename_rLung = f"{rLung_filename_base}.vtk"
                vtk_filename_Aseed = f"{Aseed_filename_base}.vtk"
                output_filename_rLung = output_path_seg + vtk_filename_rLung
                output_filename_Aseed = output_path_seg + vtk_filename_Aseed
                lung_screenshot_filename = "Lung-Mask.png"
                print("yay")
                # This line creates Models Forlder if not already made
                if not os.path.exists(output_path_seg):
                    os.makedirs(output_path_seg)

                data, header = nrrd.read(output_nrrd_path_mask)
                data[data == 17] = 3
                data[data == 5] = 3
                print("yay")
                # Load the .nrrd segmentation for Vseed
                nrrd_reader_rLung = vtk.vtkNrrdReader()
                nrrd_reader_rLung.SetFileName(output_nrrd_path_mask)
                nrrd_reader_rLung.Update()

                vtk_image_data_rLung = vtk.vtkImageData()
                vtk_image_data_rLung.DeepCopy(nrrd_reader_rLung.GetOutput())

                # Define scalar value for "Vseed" segment
                rLung_scalar_value = 3

                # Create a Marching Cubes filter for "Vseed" segment
                contour_rLung = vtk.vtkMarchingCubes()
                contour_rLung.SetInputData(vtk_image_data_rLung)
                contour_rLung.SetValue(0, rLung_scalar_value)
                contour_rLung.Update()
                print("yay")
                # Write the "Vseed" mesh to a .vtk file
                vtk_writer_rLung = vtk.vtkPolyDataWriter()
                vtk_writer_rLung.SetFileName(output_filename_rLung)
                vtk_writer_rLung.SetInputData(contour_rLung.GetOutput())
                vtk_writer_rLung.Write()
                print("Lung conversion complete.")

                #CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
                screenshot_output = output_path_seg + case_number + lung_screenshot_filename
                mlab.figure(bgcolor=(1, 1, 1))
                srcrLung = mlab.pipeline.open(output_filename_rLung)
                srcAseed = mlab.pipeline.open(output_filename_Aseed)

                # Create a surface visualization of the data with blue color and 50% opacity
                surfaceR = mlab.pipeline.surface(srcrLung, color=(128/255, 174/255, 128/255), opacity=.15)
                surfaceAseed = mlab.pipeline.surface(srcAseed, color=(0, 151/255, 206/255), opacity=1)

                x, y, z = center[0], center[1], center[2]
                ypt = y + 600

                camera_focal_point = np.array([x, y, z])
                camera_position = np.array([x,ypt,z])

                print("Camera Position", camera_position)
                print("Camera Focal Point", camera_focal_point)

                # Set the camera parameters
                # Write code to calculate this
                mlab.gcf().scene.camera.position = [x, ypt, z]
                mlab.gcf().scene.camera.focal_point = [x, y, z]
                mlab.gcf().scene.camera.view_up = [0, 0, 1]

                print("Origin: ", origin)
                print("Dimensions: ", dimensions)
                print("Center: ", center)

                # Attach the callback to the interaction event

                magnification_factor = 5
                mlab.savefig(screenshot_output, magnification=magnification_factor)
                
                
                print("Process Completed")

            else:
                print("One or more files were not found for", folder)

            
           
            